In [1]:
# install Cornac and PyTorch (VBPR model implementation uses PyTorch)
!pip3 install cornac torch>=0.4.1

In [2]:
import cornac
from cornac.datasets import tradesy
from cornac.data import ImageModality
from cornac.eval_methods import RatioSplit
import pandas as pd
import numpy as np
from cornac.data import Reader
from cornac.hyperopt import Discrete, Continuous
from cornac.hyperopt import GridSearch, RandomSearch


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# VBPR extends Bayesian Personalized Randing to leverage item visual features (extracted from product images using CNN)
# The necessary data can be loaded as follows
reader=Reader()

feedback= reader.read("/content/drive/MyDrive/MITB/CS608/Project 2/amazon_feedback_full_1.2m.csv",sep=',',skip_lines=1)

In [5]:
feedback[:5]

[('A2MOB79WI93FDN', '6040972467', 5.0),
 ('A16O44MQ3ZS7CL', '6040972467', 3.0),
 ('A14ZLDIG0XD6AL', '6040972467', 3.0),
 ('AZD378NGJIDQH', '6040972467', 5.0),
 ('A339XOY8DU2UWE', '6040972467', 3.0)]

In [6]:
len(feedback)

1287045

In [7]:
df=pd.read_csv('/content/drive/MyDrive/MITB/CS608/Project 2/full_item_feats2.csv')

df.head()

,asin,feat
0,6040596368,"[0.1650523888, 0.524156368, 0.2667483240000000..."
1,6040972467,"[0.4050605, 0.33036426, 0.2321943, 0.15210894,..."
2,6040974753,"[0.33511356, 0.34719497, 0.115295, 0.08025931,..."
3,9789892837,"[0.052969208625, 0.41584283375000003, 0.255046..."
4,B00008L1ST,"[0.07350600350000001, 0.17075585066666668, 0.0..."


In [8]:
item_ids=df['asin'].tolist()
item_ids[:5]

['6040596368', '6040972467', '6040974753', '9789892837', 'B00008L1ST']

In [9]:
len(item_ids)

42951

In [10]:
def floatit(x):
  return [float(i) for i in x]

In [11]:
features_=df['feat'].apply(lambda x: x[1:-1].split(','))
features=np.asarray(features_.apply(floatit).tolist())

In [12]:
features[:5]

array([[0.16505239, 0.52415637, 0.26674832, ..., 0.38159088, 0.29361678,
        0.1017647 ],
       [0.4050605 , 0.33036426, 0.2321943 , ..., 0.22281997, 0.2165342 ,
        0.23516575],
       [0.33511356, 0.34719497, 0.115295  , ..., 0.1206347 , 0.33069977,
        0.11935604],
       [0.05296921, 0.41584283, 0.25504655, ..., 0.28925593, 1.24776427,
        0.06325168],
       [0.073506  , 0.17075585, 0.06438219, ..., 0.17174458, 0.94780602,
        0.15769453]])

In [13]:
len(features)

42951

In [14]:
item_image_modality = ImageModality(features=features, ids=item_ids, normalized=True)

In [15]:
# Define an evaluation method to split feedback into train and test sets
ratio_split = RatioSplit(
    data=feedback,
    test_size=0.15,
    val_size=0.15,
    rating_threshold=1,
    exclude_unknowns=True,
    verbose=True,
    item_image=item_image_modality,
)

rating_threshold = 1.0
exclude_unknowns = True


/usr/local/lib/python3.7/dist-packages/cornac/data/dataset.py:361: UserWarning: 13586 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)


---
Training data:
Number of users = 435526
Number of items = 41570
Number of ratings = 887345
Max rating = 5.0
Min rating = 1.0
Global mean = 4.1


/usr/local/lib/python3.7/dist-packages/cornac/data/dataset.py:361: UserWarning: 541 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)


---
Test data:
Number of users = 110918
Number of items = 25403
Number of ratings = 144083
Number of unknown users = 0
Number of unknown items = 0


/usr/local/lib/python3.7/dist-packages/cornac/data/dataset.py:361: UserWarning: 574 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)


---
Validation data:
Number of users = 110330
Number of items = 25160
Number of ratings = 143840
---
Total users = 435526
Total items = 41570


In [16]:
# Instantiate evaluation measures
eval_metrics = [
  cornac.metrics.AUC,
  cornac.metrics.Recall(k=20),
  cornac.metrics.NDCG(k=20),
  cornac.metrics.NCRR(k=20),
  cornac.metrics.Recall(k=50),
  cornac.metrics.NDCG(k=50),
  cornac.metrics.NCRR(k=50),
]

recall = cornac.metrics.Recall(k=20)

In [ ]:
# Instantiate VBPR
vbpr = cornac.models.VBPR(
    k=500,
    k2=500,
    n_epochs=50,
    batch_size=256,
    learning_rate=0.001,
    lambda_w=1,
    lambda_b=0.3,
    lambda_e=0.0,
    use_gpu=True,
)


# Wrap model inside GridSearch along with the searching space
# vbpr_search = GridSearch(
#     model=vbpr,
#     space=[
#         Discrete("k", [500]),
#         Discrete("k2", [500]),
#         Discrete("n_epochs", [15]),
#         Discrete("batch_size", [256]),
#         Discrete("learning_rate", [0.001]),
#         # Discrete("k", np.linspace(10, 100, 5)),
#         # Discrete("learning_rate", np.linspace(0.001,0.002, 2)),
#         # Discrete("lambda_b", np.linspace(0.3, 0.4, 2)),
#         # Discrete("lambda_e", np.linspace(0.0, 0.05, 2)),
#     ],
#     metric=recall,
#     eval_method=ratio_split,
# )

In [ ]:
# Evaluation
cornac.Experiment(eval_method=ratio_split, models=[vbpr], metrics=eval_metrics).run()


[VBPR] Training started!



Optimization finished!

[VBPR] Evaluation started!




VALIDATION:
...
     | NCRR@20 | NCRR@50 | NDCG@20 | NDCG@50 | Recall@20 | Recall@50 |  Time (s)
---- + ------- + ------- + ------- + ------- + --------- + --------- + ---------
VBPR |  0.0475 |  0.0493 |  0.0720 |  0.0829 |    0.1543 |    0.2060 | 1432.7852

TEST:
...
     | NCRR@20 | NCRR@50 | NDCG@20 | NDCG@50 | Recall@20 | Recall@50 |  Train (s) |  Test (s)
---- + ------- + ------- + ------- + ------- + --------- + --------- + ---------- + ---------
VBPR |  0.0474 |  0.0492 |  0.0715 |  0.0823 |    0.1526 |    0.2037 | 15844.6614 | 1439.8635



In [ ]:
vbpr.save('/content/drive/MyDrive/MITB/CS608/Project 2/saved_models/vbpr_k500_epoch50')

VBPR model is saved to /content/drive/MyDrive/MITB/CS608/Project 2/saved_models/vbpr_k1000/VBPR/2021-07-18_20-29-39-207832.pkl


'/content/drive/MyDrive/MITB/CS608/Project 2/saved_models/vbpr_k1000/VBPR/2021-07-18_20-29-39-207832.pkl'

In [ ]:
vbpr.load('/content/drive/MyDrive/MITB/CS608/Project 2/saved_models/vbpr_k500_epoch15_batch256_lr0_001/VBPR/2021-07-18_14-40-00-704407.pkl', trainable=False)

In [ ]:
# Define an evaluation method to split feedback into train and test sets
ratio_split = RatioSplit(
    data=feedback[:128704],
    test_size=0.15,
    val_size=0.15,
    rating_threshold=1,
    exclude_unknowns=True,
    verbose=True,
    item_image=item_image_modality,
)

rating_threshold = 1.0
exclude_unknowns = True


/usr/local/lib/python3.7/dist-packages/cornac/data/dataset.py:361: UserWarning: 11583 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)
/usr/local/lib/python3.7/dist-packages/cornac/data/dataset.py:361: UserWarning: 345 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)
/usr/local/lib/python3.7/dist-packages/cornac/data/dataset.py:361: UserWarning: 348 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)


---
Training data:
Number of users = 49169
Number of items = 841
Number of ratings = 78509
Max rating = 5.0
Min rating = 1.0
Global mean = 4.3
---
Test data:
Number of users = 11007
Number of items = 501
Number of ratings = 13078
Number of unknown users = 0
Number of unknown items = 0
---
Validation data:
Number of users = 11158
Number of items = 524
Number of ratings = 13199
---
Total users = 49169
Total items = 841


In [ ]:
# Instantiate VBPR
vbpr = cornac.models.VBPR(
    k=100,
    k2=100,
    n_epochs=10,
    batch_size=256,
    learning_rate=0.001,
    lambda_w=1,
    lambda_b=0.3,
    lambda_e=0.0,
    use_gpu=True,
)


# Wrap model inside GridSearch along with the searching space
vbpr_search = GridSearch(
    model=vbpr,
    space=[
        Discrete("k", [500]),
        Discrete("k2", [200,250,300,350,400,450,500]),
        Discrete("n_epochs", [15]),
        Discrete("batch_size", [256]),
        Discrete("learning_rate", [0.001]),
        # Discrete("k", np.linspace(10, 100, 5)),
        # Discrete("learning_rate", np.linspace(0.001,0.002, 2)),
        # Discrete("lambda_b", np.linspace(0.3, 0.4, 2)),
        # Discrete("lambda_e", np.linspace(0.0, 0.05, 2)),
    ],
    metric=recall,
    eval_method=ratio_split,
)

In [ ]:
# Evaluation
cornac.Experiment(eval_method=ratio_split, models=[vbpr_search], metrics=eval_metrics).run()


[GridSearch_VBPR] Training started!
Evaluating: {'batch_size': 256, 'k': 500, 'k2': 200, 'learning_rate': 0.001, 'n_epochs': 15}



Optimization finished!
Evaluating: {'batch_size': 256, 'k': 500, 'k2': 250, 'learning_rate': 0.001, 'n_epochs': 15}



Optimization finished!
Evaluating: {'batch_size': 256, 'k': 500, 'k2': 300, 'learning_rate': 0.001, 'n_epochs': 15}



Optimization finished!
Evaluating: {'batch_size': 256, 'k': 500, 'k2': 350, 'learning_rate': 0.001, 'n_epochs': 15}



Optimization finished!
Evaluating: {'batch_size': 256, 'k': 500, 'k2': 400, 'learning_rate': 0.001, 'n_epochs': 15}



Optimization finished!
Evaluating: {'batch_size': 256, 'k': 500, 'k2': 450, 'learning_rate': 0.001, 'n_epochs': 15}



Optimization finished!
Evaluating: {'batch_size': 256, 'k': 500, 'k2': 500, 'learning_rate': 0.001, 'n_epochs': 15}



Optimization finished!
Best parameter settings: {'batch_size': 256, 'k': 500, 'k2': 350, 'learning_rate': 0.001, 'n_epochs': 15}
Recall@20 = 0.9040

[GridSearch_VBPR] Evaluation started!




VALIDATION:
...
                | NCRR@20 | NCRR@50 | NDCG@20 | NDCG@50 | Recall@20 | Recall@50 | Time (s)
--------------- + ------- + ------- + ------- + ------- + --------- + --------- + --------
GridSearch_VBPR |  0.4962 |  0.4971 |  0.5967 |  0.6025 |    0.9040 |    0.9318 |  14.5314

TEST:
...
                | NCRR@20 | NCRR@50 | NDCG@20 | NDCG@50 | Recall@20 | Recall@50 | Train (s) | Test (s)
--------------- + ------- + ------- + ------- + ------- + --------- + --------- + --------- + --------
GridSearch_VBPR |  0.4896 |  0.4907 |  0.5908 |  0.5972 |    0.8983 |    0.9286 |  724.6882 |  15.7444



In [ ]:
# Define an evaluation method to split feedback into train and test sets
ratio_split = RatioSplit(
    data=feedback,
    test_size=0.15,
    val_size=0.15,
    rating_threshold=1,
    exclude_unknowns=True,
    verbose=True,
    item_image=item_image_modality,
)

rating_threshold = 1.0
exclude_unknowns = True


/usr/local/lib/python3.7/dist-packages/cornac/data/dataset.py:361: UserWarning: 13676 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)


---
Training data:
Number of users = 435734
Number of items = 41579
Number of ratings = 887255
Max rating = 5.0
Min rating = 1.0
Global mean = 4.1


/usr/local/lib/python3.7/dist-packages/cornac/data/dataset.py:361: UserWarning: 567 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)


---
Test data:
Number of users = 110913
Number of items = 25296
Number of ratings = 144314
Number of unknown users = 0
Number of unknown items = 0


/usr/local/lib/python3.7/dist-packages/cornac/data/dataset.py:361: UserWarning: 516 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)


---
Validation data:
Number of users = 110759
Number of items = 25299
Number of ratings = 144149
---
Total users = 435734
Total items = 41579


In [20]:
# Instantiate VBPR
vbpr = cornac.models.VBPR(
    k=1000,
    k2=400,
    n_epochs=15,
    batch_size=256,
    learning_rate=0.001,
    lambda_w=1,
    lambda_b=0.3,
    lambda_e=0.0,
    use_gpu=True,
)

In [21]:
# Evaluation
cornac.Experiment(eval_method=ratio_split, models=[vbpr], metrics=eval_metrics).run()


[VBPR] Training started!



Optimization finished!

[VBPR] Evaluation started!




VALIDATION:
...
     | NCRR@20 | NCRR@50 | NDCG@20 | NDCG@50 | Recall@20 | Recall@50 |  Time (s)
---- + ------- + ------- + ------- + ------- + --------- + --------- + ---------
VBPR |  0.0410 |  0.0428 |  0.0633 |  0.0739 |    0.1384 |    0.1888 | 2150.2548

TEST:
...
     | NCRR@20 | NCRR@50 | NDCG@20 | NDCG@50 | Recall@20 | Recall@50 | Train (s) |  Test (s)
---- + ------- + ------- + ------- + ------- + --------- + --------- + --------- + ---------
VBPR |  0.0427 |  0.0444 |  0.0654 |  0.0759 |    0.1418 |    0.1918 | 6884.9364 | 2179.3948



In [22]:
vbpr.save('/content/drive/MyDrive/MITB/CS608/Project 2/saved_models/vbpr_k1000_k2_400')

VBPR model is saved to /content/drive/MyDrive/MITB/CS608/Project 2/saved_models/vbpr_k1000_k2_400/VBPR/2021-07-22_04-59-39-036456.pkl


'/content/drive/MyDrive/MITB/CS608/Project 2/saved_models/vbpr_k1000_k2_400/VBPR/2021-07-22_04-59-39-036456.pkl'

In [23]:
train_set = vbpr.train_set
uid2raw = {v:k for k,v in train_set.uid_map.items()}
iid2raw = {v:k for k,v in train_set.iid_map.items()}

item_id2idx = vbpr.train_set.iid_map
item_idx2id = list(vbpr.train_set.item_ids)
img_features = vbpr.train_set.item_image.features

In [ ]:
from collections import defaultdict

In [40]:
UID = 'A2522ZTCKCJZQT'
UIDX = train_set.uid_map[UID]
TOPK = 20

recommendations, scores = vbpr.rank(UIDX)
print(f"\nTOP {TOPK} RECOMMENDATIONS for {UID}")

rec_df = defaultdict(list)
for i in recommendations[:TOPK]:
  cf_contrib = vbpr.beta_item[i] + np.dot(vbpr.gamma_item[i], vbpr.gamma_user[UIDX])
  vis_contrib = vbpr.visual_bias[i] + np.dot(vbpr.theta_item[i], vbpr.theta_user[UIDX])
  rec_df["Recommendation"].append(item_idx2id[i])
  rec_df["Score"].append(scores[i])
  rec_df["CF Contribution"].append(cf_contrib)
  rec_df["Visual Contribution"].append(vis_contrib)
pd.DataFrame(rec_df)


TOP 20 RECOMMENDATIONS for A2522ZTCKCJZQT


,Recommendation,Score,CF Contribution,Visual Contribution
0,B00XWF727Y,17.070988,-0.692016,17.763004
1,B01FMA9VIE,16.337269,-1.012713,17.349981
2,B00Y1AU640,15.986072,-0.857707,16.843779
3,B01EWAXB2M,14.967872,-0.828396,15.796267
4,B00KB2WYG0,14.640557,0.180396,14.460161
5,B00SWH0E5Y,14.412086,-0.912401,15.324486
6,B00XWZ51EA,14.317208,0.150577,14.166632
7,B00YMOQTUU,14.087952,-0.936630,15.024582
8,B019FATCTA,14.072140,-0.572813,14.644953
9,B01D67ZNNC,13.858322,-0.465446,14.323768


In [41]:
UID = 'A2PACJHFKI74Y3'
UIDX = train_set.uid_map[UID]
TOPK = 20

recommendations, scores = vbpr.rank(UIDX)
print(f"\nTOP {TOPK} RECOMMENDATIONS for {UID}")

rec_df = defaultdict(list)
for i in recommendations[:TOPK]:
  cf_contrib = vbpr.beta_item[i] + np.dot(vbpr.gamma_item[i], vbpr.gamma_user[UIDX])
  vis_contrib = vbpr.visual_bias[i] + np.dot(vbpr.theta_item[i], vbpr.theta_user[UIDX])
  rec_df["Recommendation"].append(item_idx2id[i])
  rec_df["Score"].append(scores[i])
  rec_df["CF Contribution"].append(cf_contrib)
  rec_df["Visual Contribution"].append(vis_contrib)
pd.DataFrame(rec_df)


TOP 20 RECOMMENDATIONS for A2PACJHFKI74Y3


,Recommendation,Score,CF Contribution,Visual Contribution
0,B01F5XQ0DC,11.047301,-0.564193,11.611494
1,B00LQA5XX2,10.990538,0.805344,10.185194
2,B00YAZ80HG,10.721180,0.045886,10.675294
3,B00HTM0LJM,10.608603,-1.014025,11.622627
4,B001HTU562,10.573609,-0.975145,11.548755
5,B01BPX7CU6,10.544291,0.001416,10.542873
6,B017F067CW,10.254682,0.793958,9.460724
7,B00J9Y7IAI,10.178699,1.175990,9.002708
8,B0147L2K50,10.136951,2.211661,7.925292
9,B017IK5MV6,10.102419,-0.835271,10.937690


In [44]:
UID = 'A2OSLOUKH5VWGL'
UIDX = train_set.uid_map[UID]
TOPK = 20

recommendations, scores = vbpr.rank(UIDX)
print(f"\nTOP {TOPK} RECOMMENDATIONS for {UID}")

rec_df = defaultdict(list)
for i in recommendations[:TOPK]:
  cf_contrib = vbpr.beta_item[i] + np.dot(vbpr.gamma_item[i], vbpr.gamma_user[UIDX])
  vis_contrib = vbpr.visual_bias[i] + np.dot(vbpr.theta_item[i], vbpr.theta_user[UIDX])
  rec_df["Recommendation"].append(item_idx2id[i])
  rec_df["Score"].append(scores[i])
  rec_df["CF Contribution"].append(cf_contrib)
  rec_df["Visual Contribution"].append(vis_contrib)
pd.DataFrame(rec_df)


TOP 20 RECOMMENDATIONS for A2OSLOUKH5VWGL


,Recommendation,Score,CF Contribution,Visual Contribution
0,B01B5BCWIK,29.708738,-1.246146,30.954884
1,B005MZHZQ0,26.102215,-1.169888,27.272102
2,B019WVMRYY,22.824528,-0.995026,23.819553
3,B00CHNMU1U,21.233126,0.216193,21.016932
4,B00N17UIAG,20.805637,-0.944238,21.749876
5,B00BQ9G558,20.512989,-0.977108,21.490097
6,B00BQ8HQBG,20.349445,-0.986210,21.335655
7,B00M3AB85G,20.026449,0.161745,19.864704
8,B00CI7ERWK,19.802387,-0.969190,20.771578
9,B00QTSG6Q4,19.780844,-0.932974,20.713818


In [45]:
UID = 'A1HV6KKVTGMCGE'
UIDX = train_set.uid_map[UID]
TOPK = 20

recommendations, scores = vbpr.rank(UIDX)
print(f"\nTOP {TOPK} RECOMMENDATIONS for {UID}")

rec_df = defaultdict(list)
for i in recommendations[:TOPK]:
  cf_contrib = vbpr.beta_item[i] + np.dot(vbpr.gamma_item[i], vbpr.gamma_user[UIDX])
  vis_contrib = vbpr.visual_bias[i] + np.dot(vbpr.theta_item[i], vbpr.theta_user[UIDX])
  rec_df["Recommendation"].append(item_idx2id[i])
  rec_df["Score"].append(scores[i])
  rec_df["CF Contribution"].append(cf_contrib)
  rec_df["Visual Contribution"].append(vis_contrib)
pd.DataFrame(rec_df)


TOP 20 RECOMMENDATIONS for A1HV6KKVTGMCGE


,Recommendation,Score,CF Contribution,Visual Contribution
0,B01FQS7RSS,6.645053,2.757894,3.887160
1,B01FQT1DJG,6.534947,3.047158,3.487789
2,B019MBG8FS,6.246087,0.521592,5.724494
3,B01FYI49BS,5.948618,-0.947471,6.896089
4,B019M7DU94,5.858365,1.651960,4.206405
5,B01H3CSW0C,5.856915,1.687457,4.169458
6,B01FVZH3Q2,5.665293,1.435827,4.229466
7,B01E7EO1T0,5.562914,1.948191,3.614722
8,B0111WY8MQ,5.545915,1.195125,4.350790
9,B01G3GK0MW,5.505595,1.718000,3.787596
